# 04: Aggregate Appointment Data

In addition to the cleaning done in the previous notebook, we can also aggregate the appointment data to one line per patient. 

We do so here, creating some calculated measures of the appointments and doctors seen.

In [ ]:
%run 00_config

In [ ]:
'''
df = spark.sql("SELECT * FROM coffee_lakehouse.appointment_df_clean")
display(df)
'''

In [ ]:
# Read the data from the lakehouse
df = spark.sql(f"""
SELECT
    patient_id,
    -- appointment_date
    COUNT(DISTINCT(appointment_date)) AS count_appointments,
    MAX(appointment_date) AS latest_appointment,
    MIN(appointment_date) AS earliest_appointment,
    -- doctor_seen
    COUNT(DISTINCT(doctor_seen)) AS count_doctors_seen
FROM 
    {config['lakehouse_name']}.appointment_df_clean
GROUP BY
    patient_id
""")

In [ ]:
# Save the data to the lakehouse
(
df
    .write
    .mode('overwrite')
    .format('delta')
    .option('overwriteSchema', 'true')
    .save(f"{config['lakehouse_path']}/appointment_df_agg")
)